In [13]:
from maggma.stores.advanced_stores import MongograntStore
from maggma.stores.advanced_stores import Sort
from typing import List
from pathlib import Path
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import timedelta, date, datetime
from monty.json import MontyDecoder
from datetime import timedelta
# configuration stuff
from sys import platform
import maggma
if platform == "linux" or platform == "linux2":
    import plotly.io as pio
    pio.orca.config.use_xvfb = True
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
import plotly

In [28]:
renderer = "notebook" # change to pdf for live viewing

In [29]:
gdrive_mongo_store = MongograntStore(mongogrant_spec="rw:knowhere.lbl.gov/mp_core_mwu",
                                         collection_name="gdrive")
gdrive_mongo_store.connect()

tasks_mongo_store = MongograntStore(mongogrant_spec="ro:mongodb04.nersc.gov/mp_emmet_prod",
                                        collection_name="tasks")
tasks_mongo_store.connect()

In [30]:
def clear_uploaded_fields():
    mongo_store = MongograntStore(mongogrant_spec="rw:knowhere.lbl.gov/mp_core_mwu",
                                         collection_name="gdrive")
    gdrive_mongo_store.connect()
    cursor = mongo_store.query(criteria=
        {"nomad_updated": {"$ne": None}}
    )
    new_objs = []
    for obj in cursor:
        obj["nomad_updated"] = None
        obj["nomad_upload_id"] = None
        new_objs.append(obj)
    mongo_store.update(docs=new_objs, key="task_id")
    gdrive_mongo_store.close()
    print("Cleared")

In [31]:
import plotly.graph_objects as go

df = pd.DataFrame()
df["title"] = np.array(["Total in Gdrive", "Total Tasks"])
df["count"] = np.array([gdrive_mongo_store.count(), tasks_mongo_store.count()])
fig = px.pie(df, values='count', names='title', title="Tasks & GDrive")
fig.show(renderer=renderer)

print("WARNING: This Pie chart might not reflect the actual progress since there are tasks that belong to an deprecated material")

In [32]:
df = pd.DataFrame()
df["title"] = np.array(["Total in Gdrive","Total in NOMAD" ])
df["count"] = np.array([gdrive_mongo_store.count(), 
                        gdrive_mongo_store.count(criteria={"nomad_updated": {"$ne": None}})])
fig = px.pie(df, values='count', names='title', title="GDrive & NOMAD")
fig.show(renderer=renderer)





In [19]:
df = pd.DataFrame()
df["title"] = np.array(["Success","Failed" ])
df["count"] = np.array([gdrive_mongo_store.count(criteria={"error": {"$eq": None}}), 
                        gdrive_mongo_store.count(criteria={"error": {"$ne": None}})])
fig = px.pie(df, values='count', names='title', title="GDrive Upload Status")
fig.show(renderer=renderer)



In [20]:
all_content = gdrive_mongo_store.query({},{"file_size":1})
size = 0
for c in all_content:
    size += c["file_size"]
print(f"{size} bytes = {size*1e-6} mb = {size*1e-9} gb")

712280123385 bytes = 712280.1233849999 mb = 712.280123385 gb


In [21]:
def find_dates_btw(start_dt, end_dt):
    """
    find the number of dates between start date and end date
    """
    def daterange(date1, date2):
        if date1 is None: date1 = date2
        if date2 is None: date2 = date1
        for n in range(int((date2 - date1).days)+1):
            yield date1 + timedelta(n)
    dates = []
    for dt in daterange(start_dt, end_dt+timedelta(days=1)):
        date_format = dt.date()
        dates.append(datetime(date_format.year, date_format.month, date_format.day))
    return dates

def find_earliest_date(store, field):
    """
    find the earliest record date
    """
    return list(store.query(criteria={}, sort={field:maggma.core.store.Sort.Ascending}, limit=1))[0][field]

def find_latest_date(store, field):
    """
    find the latest_record date
    """
    return list(store.query(criteria={}, sort={field:maggma.core.store.Sort.Descending}, limit=1))[0][field]

In [22]:
def make_time_series_data(field_name):
    """
    Find all time series data for that field, put them in buckets of dates.
    """
    dates = find_dates_btw(find_earliest_date(gdrive_mongo_store, field_name), 
                           find_latest_date(gdrive_mongo_store, field_name))
    # last_updated 
    result = dict()
    for i in range(len(dates)):
        if i == 0:
            result[dates[i]] = 0
        else:
            c = gdrive_mongo_store.count(criteria={field_name: {"$lte": dates[i]}})
            result[dates[i]] = c
    return result

In [23]:
last_updated_data = make_time_series_data("last_updated")
Xs = set(last_updated_data.keys())
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(last_updated_data.keys()), y=list(last_updated_data.values()),
                    mode='lines+markers',
                    name='last_updated'))

# add features
fig.update_layout(
    title="GDrive Upload Status",
    xaxis_title="Time",
    yaxis_title="# Submission",
    font=dict(
        family="Franklin Gothic",
        size=14,
        color="#0d0d0d"
    ),   
    yaxis_type="log",
)
fig.show(renderer=renderer)